# Admin Startup

The purpose of this notebook is to show how to start an admin client to operate an FL experiment with a server and at least one client started.

## Prerequisites
- The [Startup notebook](Startup.ipynb) has been run successfully.
- A server and at least one client has been started.

### Check the Config Folder

The config folder should be in the `transfer/` directory:

In [1]:
import os
from IPython.display import HTML
from multiprocessing import Process

workspace = "demo_workspace/"
admin_name = "admin@nvidia.com"

transfer_path = os.path.join(workspace, admin_name, "transfer")
os.listdir(transfer_path)

['spleen_example']

### Edit Docker Script

Before starting the docker script, you need to edit it to ensure that the environments (such as dataset, GPUs, memory, ...) meet your requirement.

[Please click here to check the admin@nvidia.com script](demo_workspace/admin@nvidia.com/startup/docker.sh)

For this experiment, please modify the script in order to use the host network:

```
#!/usr/bin/env bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
# docker run script for FL admin
# to use host network, use line below
NETARG="--net=host"
# Admin clients do not need to open ports, so the following line is not needed.
#NETARG="-p 8003:8003"
DOCKER_IMAGE=monai_nvflare:latest
echo "Starting docker with $DOCKER_IMAGE"
docker run --rm -it --name=fladmin -v $DIR/..:/workspace/ -w /workspace/ --ipc=host $NETARG $DOCKER_IMAGE /bin/bash
```

### Start Admin Docker

In [2]:
admin_startup_path = os.path.join(workspace, admin_name, "startup")
cmd = admin_startup_path + "/docker.sh"


def run_admin():
    cmd = admin_startup_path + "/docker.sh"
    print("running cmd " + cmd)
    !$cmd


p1 = Process(target=run_admin)

p1.start()

running cmd demo_workspace/admin@nvidia.com/startup/docker.sh
Starting docker with monai_nvflare:latest

== PyTorch ==

NVIDIA Release 21.08 (build 26011915)
PyTorch Version 1.10.0a0+3fd9dcf

Container image Copyright (c) 2021, NVIDIA CORPORATION.  All rights reserved.

Copyright (c) 2014-2021 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert, Leon Bottou, Iain Melvin, Jason Weston)
Copyright (c) 2006      Idiap Research Institute (Samy Bengio)
Copyright (c) 2001-2004 Idiap Research Institute (Ronan Collobert, Samy Bengio, Johnny Mariethoz)
Copyright (c) 2015      Google Inc.
Copyright (c) 2015      Yangqing Jia
Copyright (c) 2013-2016 The Caffe contributors
All rights reserved.

NVIDIA De

### Check Started Containers

In [3]:
!docker ps -a

CONTAINER ID   IMAGE                  COMMAND                  CREATED              STATUS              PORTS     NAMES
3de0f6690d8e   monai_nvflare:latest   "/usr/local/bin/nvid…"   6 seconds ago        Up 6 seconds                  fladmin
96c8f9406303   monai_nvflare:latest   "/usr/local/bin/nvid…"   52 seconds ago       Up 52 seconds                 org1-a
6b5d9eefc699   monai_nvflare:latest   "/usr/local/bin/nvid…"   52 seconds ago       Up 52 seconds                 org1-b
08492335d003   monai_nvflare:latest   "/usr/local/bin/nvid…"   About a minute ago   Up About a minute             flserver


### Start Admin

To start an admin, you should:

- open a terminal and enter the container named `fladmin`.
- run `fl_admin.sh` under `startup/`.
- input admin name `admin@nvidia.com`.

In [4]:
HTML('<a href="", data-commandlinker-command="terminal:create-new"> Open a new terminal</a>')

You can accomplish these steps by running:

```
docker exec -it fladmin bash
cd startup/
bash fl_admin.sh
admin@nvidia.com
```

### Prepare for the experiment

You need to execute the following steps to prepare for the experiment:

- upload pipeline config folder
- set FL training number
- deploy the folder to client(s) and server

The commands can be:
```
upload_app spleen_example
set_run_number 1
deploy_app spleen_example server
deploy_app spleen_example client
```

Now, let's check if the folder has been distributed into the server and all client(s):

In [5]:
run_file = 'run_1'

for name in ['server', 'org1-a', 'org1-b']:
    path = os.path.join(workspace, name, run_file)
    if os.path.exists(path):
        print("config files on {}: {}".
              format(name, os.listdir(path)))
        print(" ")

config files on server: ['mmar_server', 'mmar_org1-a', 'mmar_org1-b']
 
config files on org1-a: ['mmar_org1-a']
 
config files on org1-b: ['mmar_org1-b']
 


This example prepares two different data list files: `dataset_part1.json` and `dataset_part2.json`, and they have the same validation set and totally different training set. The default file used in `spleen_example/config_train.json` is `dataset_part1.json`. Therefore, if you want to let two clients train on different data, you can switch to use `dataset_part2.json` for `org1-b`.

[Link to org1-a config](demo_workspace/org1-a/run_1/mmar_org1-a/config/config_train.json)

[Link to org1-b config](demo_workspace/org1-b/run_1/mmar_org1-b/config/config_train.json)

### Start Training

Now, you can start training with:
```
start_app server
start_app client
```
You can check the status by running:
```
check_status server
check_status client
```

### Stop Training

You can stop training for the server and/or client(s) by running:
```
abort client
abort server
```
If you only want to stop org1-b, you can use:
```
abort client org1-b
```

### Start Training (multi-gpus)

If you would like to use multiple gpus, you can start training with n gpus (where n is the number of gpus) by running:
```
start_mgpu client n
```

The default `multi_gpu` flag is `False` in `spleen_example/config/config_train.json`, if you need to use multiple gpus, you have to change it before the `Startup` step.

### Shutdown

You can shutdown the experiment for the server and/or client(s) by running:

`shutdown client` and/or `shutdown server`

If you only want to shutdown a specific client, you can specify the client in the command like follows:
```
shutdown client fl_clients_2
```
This command will kill the client/server connection, and this command will need input of the admin name for confirmation. If you need to shutdown the server, all active clients need to be shutdown first.

After the shutdown commands, you are safe to shutdown `Startup.ipynb` to stop all containers.

### Other Commands

Please type `?` to learn more about all commands.

### Next Steps

You have now started the admin client and learnt the commands to control your FL experiment. You're now ready to create your own FL experiment!